In [1]:
import duckdb
import pandas as pd

%load_ext sql

In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
%sql duckdb:///:memory:

In [4]:
#Loading provided data
users=pd.read_json('users.json', lines=True)
receipts=pd.read_json('receipts.json',lines=True)
brands=pd.read_json('brands.json',lines=True)

In [5]:
#Reformatting some of the columns to extract the values
def extract_val(x,key):
    if isinstance(x, dict):
        return x.get(key)
    else:
        return None

users['_id']=users['_id'].apply(lambda x: extract_val(x, '$oid'))
users['createdDate']=users['createdDate'].apply(lambda x: extract_val(x, '$date'))
users['lastLogin'] = users['lastLogin'].apply(lambda x: extract_val(x, '$date'))
receipts['_id']=receipts['_id'].apply(lambda x: extract_val(x, '$oid'))
receipts['createDate']=receipts['createDate'].apply(lambda x: extract_val(x, '$date'))
receipts['dateScanned']=receipts['dateScanned'].apply(lambda x: extract_val(x, '$date'))
receipts['finishedDate']=receipts['finishedDate'].apply(lambda x: extract_val(x, '$date'))
receipts['modifyDate']=receipts['modifyDate'].apply(lambda x: extract_val(x, '$date'))
receipts['pointsAwardedDate']=receipts['pointsAwardedDate'].apply(lambda x: extract_val(x, '$date'))
receipts['purchaseDate']=receipts['purchaseDate'].apply(lambda x: extract_val(x, '$date'))
brands['_id']=brands['_id'].apply(lambda x: extract_val(x, '$oid'))

In [6]:
#Extracting barcode from receipts['rewardsReceiptItemList'] to demonstrate some data quality issues
def extract_barcodes(json_list):
    if not isinstance(json_list, list):
        return []
    return [item['barcode'] for item in json_list if 'barcode' in item]

receipts['barcode'] =receipts['rewardsReceiptItemList'].apply(extract_barcodes)
receipts= receipts.explode('barcode')

#### Data Quality Issue #1: Incomplete Receipt Data
Not every receipt has a barcode that can be matched to the brand table to identify what was purchased. When barcode is not found, it is assigned the value '4011' and some barcodes are simply null.

In [7]:
%%sql
select *
from receipts
where barcode is NULL or barcode='4011'
limit 10;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,barcode
0,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.0,1.609632e+12,1.0,"[{'key': ['needsFetchReview', 'partnerItemId',...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,None
1,5ff1e1cd0a720f052300056f,5.0,All-receipts receipt bonus,1609687501000,1609687501000,1.609688e+12,1609687502000,1.609688e+12,5.0,1.609688e+12,1.0,"[{'key': ['brandCode', 'competitorRewardsGroup...",FINISHED,2.23,5ff1e194b6a9d73a3a9f1052,None
2,5ff1e1e90a7214ada1000569,NaN,None,1609687529000,1609687529000,NaN,1609687529000,NaN,NaN,NaN,0.0,"[{'key': ['needsFetchReview', 'needsFetchRevie...",FLAGGED,0.00,5ff1e1e9b6a9d73a3a9f10f6,None
3,5ff3416f0a7214ada1000576,25.0,COMPLETE_NONPARTNER_RECEIPT,1609777519000,1609777519000,1.609778e+12,1609777519000,1.609778e+12,25.0,1.609718e+12,1.0,"[{'key': ['description', 'discountedItemPrice'...",FINISHED,28.57,5c3388caea88e15513a95069,None
4,5ff3416d0a7214ada1000575,25.0,COMPLETE_NONPARTNER_RECEIPT,1609777517000,1609777517000,1.609778e+12,1609777517000,1.609778e+12,25.0,1.609718e+12,1.0,"[{'key': ['description', 'discountedItemPrice'...",FINISHED,28.57,5c3388caea88e15513a95069,None
5,5ff4a4ca0a7214ada10005d0,750.0,"Receipt number 1 completed, bonus point schedu...",1609868490000,1609868490000,NaN,1609868492000,NaN,750.0,1.599955e+12,2.0,"[{'key': ['description', 'discountedItemPrice'...",REJECTED,34.96,5fbc35711d967d1222cbfefc,None
6,5ff475820a7214ada10005cf,NaN,None,1609856386000,1609856386000,NaN,1609856386000,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5a43c08fe4b014fd6b6a0612,None
7,5ff4ce3c0a720f05230005c4,NaN,None,1609879100000,1609879100000,NaN,1609879100000,NaN,NaN,NaN,0.0,"[{'key': ['needsFetchReview', 'needsFetchRevie...",FLAGGED,0.00,5ff4ce3cc1e2d0121a9b2fba,None
8,5ff4a80e0a720f05230005c2,750.0,"Receipt number 1 completed, bonus point schedu...",1609869326000,1609869326000,1.609869e+12,1609869329000,1.609869e+12,750.0,1.609286e+12,2.0,"[{'key': ['description', 'discountedItemPrice'...",FINISHED,34.96,5fbc35711d967d1222cbfefc,None
9,5ff5ecb90a7214ada10005f9,NaN,None,1609952440000,1609952440000,NaN,1609952440000,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5a43c08fe4b014fd6b6a0612,None


#### Data Quality Issue #2: Missing Barcodes in Brands
In the receipts data, there are receipts that contain Doritos according to the item description with barcode value "028400642033". The brands data, however, does not have a matching barcode for Doritos. There is a completely seperate barcode value for Doritos in the brands table. I suspect this is because I was given a truncated brands file. This is one example of the issue and I strongly suspect there are more.

In [8]:
%sql select * from receipts where barcode='028400642033' limit 10;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,barcode
0,5ff618e30a7214ada10005fa,750.0,"Receipt number 1 completed, bonus point schedu...",1609963747000,1609963747000,1.609964e+12,1609963785000,1.609964e+12,793.1,1.609891e+12,16.0,"[{'key': ['description', 'discountedItemPrice'...",FINISHED,52.12,5ff616a68f142f11dd189163,028400642033
1,600373700a720f05f3000091,750.0,"Receipt number 1 completed, bonus point schedu...",1610838896000,1610838896000,1.610840e+12,1610839900000,1.610840e+12,2416.7,1.610755e+12,211.0,"[{'key': ['barcode', 'brandCode', 'description...",FINISHED,803.48,60037329e257124ec6b99eeb,028400642033
2,60023e8f0a720f05f300008b,750.0,"Receipt number 1 completed, bonus point schedu...",1610759823000,1610759823000,1.610760e+12,1610760290000,1.610760e+12,1806.4,1.609805e+12,202.0,"[{'key': ['brandCode', 'description', 'discoun...",FINISHED,776.79,60023de5fb296c121a81b955,028400642033
3,60023e8f0a720f05f300008b,750.0,"Receipt number 1 completed, bonus point schedu...",1610759823000,1610759823000,1.610760e+12,1610760290000,1.610760e+12,1806.4,1.609805e+12,202.0,"[{'key': ['brandCode', 'description', 'discoun...",FINISHED,776.79,60023de5fb296c121a81b955,028400642033
4,6002590c0a7214ad4c000082,750.0,"Receipt number 1 completed, bonus point schedu...",1610766604000,1610766604000,1.610767e+12,1610767152000,1.610767e+12,1516.9,1.610582e+12,208.0,"[{'key': ['description', 'discountedItemPrice'...",FINISHED,746.94,600258dafb296c4ef805d9b7,028400642033
5,600260210a720f05f300008f,750.0,"Receipt number 1 completed, bonus point schedu...",1610768417000,1610768417000,1.610769e+12,1610768851000,1.610769e+12,3659.4,1.610669e+12,309.0,"[{'key': ['barcode', 'brandCode', 'description...",FINISHED,1043.18,60025fe0e257124ec6b99a87,028400642033
6,600260210a720f05f300008f,750.0,"Receipt number 1 completed, bonus point schedu...",1610768417000,1610768417000,1.610769e+12,1610768851000,1.610769e+12,3659.4,1.610669e+12,309.0,"[{'key': ['barcode', 'brandCode', 'description...",FINISHED,1043.18,60025fe0e257124ec6b99a87,028400642033
7,600260210a720f05f300008f,750.0,"Receipt number 1 completed, bonus point schedu...",1610768417000,1610768417000,1.610769e+12,1610768851000,1.610769e+12,3659.4,1.610669e+12,309.0,"[{'key': ['barcode', 'brandCode', 'description...",FINISHED,1043.18,60025fe0e257124ec6b99a87,028400642033
8,600260210a720f05f300008f,750.0,"Receipt number 1 completed, bonus point schedu...",1610768417000,1610768417000,1.610769e+12,1610768851000,1.610769e+12,3659.4,1.610669e+12,309.0,"[{'key': ['barcode', 'brandCode', 'description...",FINISHED,1043.18,60025fe0e257124ec6b99a87,028400642033
9,600260210a720f05f300008f,750.0,"Receipt number 1 completed, bonus point schedu...",1610768417000,1610768417000,1.610769e+12,1610768851000,1.610769e+12,3659.4,1.610669e+12,309.0,"[{'key': ['barcode', 'brandCode', 'description...",FINISHED,1043.18,60025fe0e257124ec6b99a87,028400642033


In [9]:
%sql select * from brands where barcode='028400642033';

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode


In [10]:
%%sql
select * from brands where brandCode='DORITOS';

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,5887a372e4b02187f85cdad9,511111001119,Snacks,None,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5fbe4b0...",Doritos,1.0,DORITOS


#### Data Issue #3: Barcode is a FK but is not unique
From my review of the data, barcode should be used to join the receipt table and the brands table. In the receipt data, barcode is a common field within 'rewardsReceiptItemList'. However, the barcode column is not a unique row identifier for the table.

In [11]:
%%sql 
select barcode, 
    count(*) as row_count 
from brands 
group by barcode 
having row_count>1;

,barcode,row_count
0,511111305125,2
1,511111504788,2
2,511111004790,2
3,511111504139,2
4,511111605058,2
5,511111704140,2
6,511111204923,2


#### Data Issue #4: User Role Naming
In the documentation for this table, role should be a constant value set to 'CONSUMER'. In the user table, there are 82 rows that are set to 'fetch-staff'. Additionally, the documentation shows that 'consumer' should be capitalized.

In [12]:
%sql select role, count(*) from users group by 1;

,role,count_star()
0,consumer,413
1,fetch-staff,82


#### Data Issue #5: Multiple Rows for User IDs 
As the ID field for the user table, each ID should have a unique value. There are 65 consumer IDs that have multiple rows of data in the users table. This prevents _id from being a valid PK for the users table.

In [13]:
%%sql 
select _id, count(*) as row_count
from users 
where role = 'consumer'
group by _id 
having row_count > 1
order by row_count desc;

,_id,row_count
0,5ff5d15aeb7c7d12096d91a2,18
1,600fb1ac73c60b12049027bb,16
2,5ff1e194b6a9d73a3a9f1052,11
3,600987d77d983a11f63cfa92,9
4,5a43c08fe4b014fd6b6a0612,8
...,...,...
60,60229990b57b8a12187fe9e0,2
61,5ffcb47d04929111f6e9256c,2
62,60088d55633aab121bb8e41a,2
63,6014558767804a1228b20d00,2


#### Additional checks
If I had more time, I would check:
- More of the brand barcodes against the receipt data
- More thorough checks for columns that appear to be categories
- In the receipt data, I would explore the rewardsReceiptItemList field more and see if there is a better way to join to the brands table 
- Check all numerical fields for outliers
- Check all columns for null percentage
- Joining rate between tables